This file is based on "4.27- Link input with activations.ipynb", with updated data structure (different list structures -> list of tuples). There are also more data that are useful for making visualizations

1. Load libraries

In [1]:
from PIL import Image
import numpy as np
import os
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms

# Use IoU instead
# import re
from scipy.spatial.distance import cdist

2. Decide the procedure by grouping the different columns together

For cityscapes and synthia separately (and then concatenate):

* name, image_path, dataset

* label_path:
    * first get labels (in class format)
        * for synthia, need to transform to class format specifically
            * for calculating similarity
        * also from class to color as well, because use the color of cityscapes
            * for saving the label and getting the label path
        * get the saved path and save it in a list
    
* similar_image_paths，similar_IoU_score (Note: called IoU_score originally): finding the most similar masks from another dataset
* Generate from os list of images:
    * class distribution
        * (other_ratio, road_ratio, sidewalk_ratio, vegetation_ratio, sky_ratio, car_ratio)
    * embedding of input (originally: tsne_1, tsne_2)
        simple dimensionality reduction (simple_tsne_1,simple_tsne_2)
        and also the embedding from classification model(meaningful_tsne_1,meaningful_tsne_1)
* bottleneck_activations_embedding, prediction_path, and performance
    * save output locally, and then add path (prediction_path)
    * performance: (other_IoU, road_IoU, sidewalk_IoU, vegetation_IoU, sky_IoU, car_IoU)
        **How do you get the IoU per class (check original paper)**

**Make sure that the column names are the same as current！！！**

# name, image_path, dataset

In [3]:
# path of the folders
image_path_cityscapes = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\original_cityscapes_inputs"
image_path_synthia = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\SYNTHIA_256\image"

start = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image"
relative_img_path_cityscapes = os.path.relpath(image_path_cityscapes, start)
relative_img_path_synthia = os.path.relpath(image_path_synthia, start)


In [15]:
sample_number = 100

In [16]:
cityscapes_names= os.listdir(image_path_cityscapes)
random.seed(55)
cityscapes_names_sample = random.sample(cityscapes_names,sample_number)

synthia_names = os.listdir(image_path_synthia)
# randomly select 100 images to load in to numpy array
random.seed(55)
synthia_names_sample = random.sample(synthia_names,sample_number)

In [21]:
# Cityscapes
cityscapes_initial_info = []

for name in cityscapes_names_sample:
    image = Image.open(image_path_cityscapes+"\\"+name).convert("RGB")
    image_path = relative_img_path_cityscapes+"\\"+name
    cityscapes_initial_info.append(tuple((name,image_path,"Cityscapes")))

cityscapes_initial_info = np.array(cityscapes_initial_info)

# Worked until here:

problem: it's still not list of tuples

In [24]:
type(cityscapes_initial_info[0])

numpy.ndarray

In [ ]:
# synthia

synthia_initial_info = []

for name in synthia_names_100:
#     print(name)
#     print(resize_image_path_synthia)
#     print(resize_image_path_synthia+"\\"+name)
    image = Image.open(resize_image_path_synthia+"\\"+name).convert("RGB")
    
    synthia_100.append(np.array(image))
    synthia_image_path_100.append(relative_path_synthia+"\\"+name)
    synthia_label_path_100.append(relative_path_synthia_labels+"\\"+name)
    label = Image.open(resize_label_path_synthia+"\\"+name).convert("RGB")
    synthia_label_100_initial.append(np.array(label))

synthia_100 = np.array(synthia_100)
synthia_image_path_100 = np.array(synthia_image_path_100)
synthia_label_path_100= np.array(synthia_label_path_100)
synthia_label_100_initial = np.array(synthia_label_100_initial)

# Synthia color transformation code (for later use)

In [ ]:
synthia_colors = [[  0,   0,   0], # void
         [70,130, 180], # sky
         [70,70,70], # building
        [128, 64, 128], # road
        [244, 35, 232], # sidewalk
         [64,64,128], # fense
         [107,142,35], # vegetation	
        [153, 153, 153], # pole
        [0, 0, 142], # car
        [220, 220, 0],  # traffic sign
        [220, 20, 60], # pedestrian
        [119, 11, 32], # bicycle
        [0, 0, 230], # motorcycle
        [250,170,160], # parking-slot
        [128,64,64], # road-work
        [250,170,30], # traffic light
        [152, 251, 152], # terrain
        [255, 0, 0], # rider
        [0, 0, 70], # truck
        [0, 60, 100], # bus
        [0, 80, 100], # train
        [102, 102, 156]# wall, lanemarking
    ]


category_map = {
    0: 0,
    1: 4,
    2: 0,
    3: 1,
    4: 2,
    5: 0,
    6: 3,
    7: 0,
    8: 5,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
    18: 0,
    19: 0,
    20: 0,
    21: 0}

In [ ]:
# color to class
def color_to_class(label):
    # create new empty mask
    mask = np.zeros(shape=(label.shape[0], label.shape[1]), dtype = np.int32)
    # iterate through two dimensions
    for row in range(label.shape[0]):
        for col in range(label.shape[1]):
            a = label[row, col,:]
            # distance between this pixel and the original pixel
            d = cdist(np.array([a]),np.array(synthia_colors))
            idx = np.argmin(d)
            new_idx = category_map[idx]
            mask[row, col] = new_idx
    mask = np.reshape(mask, (mask.shape[0], mask.shape[1]))
    return mask

In [ ]:
# class to color: useful for displaying the similar images later
colors = [[  0,   0,   0],
          [128, 64, 128],# road
          [244, 35, 232], # sidewalk
          [107, 142, 35],# vegetation
          [70, 130, 180], # sky
          [0, 0, 142], # car
         ]

def class_to_color(labels):
    label_colors = np.zeros((256,256,3))
    
    for i,row in enumerate(labels):
        for j,pixel in enumerate(row):
            label_colors[i,j] = colors[pixel]
    
    return label_colors.astype(int)  # make each pixel value an integer to visualize it better

# name, image path, label path, labels (in class format)

In [ ]:
def transform_masks_synthia(labels):
    masks = []
    for label in labels:
        mask = color_to_class(label)
        masks.append(mask)
    masks = np.array(masks)

    return masks